In [215]:
import abc
import json
import addict as adt
from pydantic import validator, root_validator
from pydantic import BaseModel
import networkx as nx
from typing import Any, List, Union, Optional
from inflection import underscore, camelize, tableize

In [225]:
class Node(abc.ABC):
    def __init__(self, _node_type:str, name:Optional[str]=None, is_entity: bool=False, **attrs):
        self._node_type: str = _node_type
        # Value represents a, Optionalnything important that we need to regularly access
        self._value: Any = None
        self._parent: Union['Node'] = None
        self._is_entity = is_entity
        self._name = name
        self._attrs = attrs
        
    @property
    def name(self) -> str:
        return self._name or underscore(self.__class__.__name__)

    @property
    def props(self) -> dict:
        return self._attrs

    @property
    def is_entity(self):
        return self._is_entity

    @property
    def parent(self):
        return self._parent

    
    @parent.setter
    def parent(self, _parent: 'Node'):
        self._parent = _parent
        
    @property
    def graph_repr(self) -> adt.Dict:
        graph_repr = adt.Dict()
        graph_repr.name = self.name
        graph_repr.attrs = adt.Dict(**self._attrs)
        graph_repr.attrs.node_type = self.ntype
        if self.value:
            graph_repr.attrs.node_value = self.value
        return graph_repr

    @property
    def ntype(self) -> str:
        return underscore(self._node_type).lower()

    

    @property
    def value(self):
        return self._value

    
    @value.setter
    def value(self, _value):
        self._value = _value

    
    def add_props(self, **attrs):
        self._attrs.update(attrs)

    def __repr__(self) -> str:
        return f"{camelize(self.ntype)}({self.name}, parent={self.parent})"

    @abc.abstractmethod
    def add(self, node: 'Node'):
        raise NotImplementedError

    
    

In [232]:
class System(Node):
    def __init__(self):
        super().__init__("system", is_entity=False)
        self.value = nx.DiGraph(name=self.name)

    @property
    def net(self) -> nx.DiGraph:
        return self.value

    def get_node(self, node: 'Subsystem'):
        return self.net.nodes[node.name]

    def add(self, node: 'Subsystem', **edges):
        node.system = self
        rep = node.graph_repr
        self.net.add_node(rep.name, **rep.attrs.to_dict())
        if node.parent:
            self.net.add_edge(node.parent.name, node.name, **edges)
        

    
    def to_dict(self):
        return json.loads(nx.jit_data(self.net))



In [236]:
class Subsystem(Node):
    def __init__(self, _node_type: str, name: Optional[str]=None, is_entity: bool=False, **attrs):
        super().__init__(_node_type, name=name, is_entity=is_entity, **attrs)

    @property
    def system(self) -> System:
        if not self._system: raise AttributeError("The system hasn't been set yet")
        return self._system
    

    @system.setter
    def system(self, _system: System) -> None:
        self._system = _system
    

    def add(self, node: 'Subsystem', **edge_props:dict):

        node.parent = self
        node.system = self.system
        self.system.add(node, **edge_props)
        return node
        
    

class Entity(Subsystem):
    def __init__(self, _type, name: Optional[str]=None, **attrs):
        _ttype = _type or tableize(self.name)
        super().__init__(_ttype, is_entity=True, name=name, **attrs)


class Component(Subsystem):
    def __init__(self, _type: str = "component", name: Optional[str]=None, **attrs):
        super().__init__(_type, is_entity=False, name=name, **attrs)


class Field(Component):
    def __init__(self, field_type:str, aggregate:str="sum", **attrs):
        super().__init__("field", **attrs)
        self.add_props(field_type=field_type, aggregate=aggregate.upper())
        


    @property
    def field_type(self) -> str:
        return self.graph_repr.field_type
    

    def create(self, field_name:str):
        self._name = field_name
        return self



class Table(Entity):
    def __init__(self, table_name: str, **attrs):
        super().__init__("table", name=table_name, **attrs)

    
    @property
    def name(self) -> str:
        return tableize(self._name) or tableize(self.__class__.__name__)

In [1]:
# TODO: Create a global pattern to create table if not exist in multiple places. Can also be used for the node iteration.
main_system = System()
user_table = Table("user")
int_field = Field("INT", "MODE")
main_system.add(user_table)
user_table.add(int_field.create("age"), rel_type="field_of")
user_table.system.to_dict()

NameError: name 'System' is not defined

In [238]:
main_system.to_dict()

[{'id': 'users',
  'name': 'users',
  'data': {'node_type': 'table'},
  'adjacencies': [{'nodeTo': 'age', 'data': {'rel_type': 'field_of'}}]},
 {'id': 'age',
  'name': 'age',
  'data': {'field_type': 'INT', 'aggregate': 'MODE', 'node_type': 'field'}}]

In [220]:


Schema()

Schema(schema, parent=None)

In [4]:
class Table(Node):
    def __init__(self):
        super().__init__("table")

In [88]:
class Table(Node):
    def __init__(self):
        super().__init__("table")

In [76]:
class Program(BaseModel):
    graph: nx.DiGraph = nx.DiGraph()
    
    class Config:
        arbitrary_types_allowed = True

    def __repr__(self):
        return f"Program(node_count={self.graph.number_of_nodes()})"

In [37]:
example_graph = nx.DiGraph()

In [38]:
example_graph.add_node("price_table", typer="table")

In [61]:
example_graph.add_node("example_field", typer="field", value="integer", name="sanic")
example_graph.add_edge("price_table", "example_field")

In [62]:
def visitor_change(node:dict):
    _typer = node['typer']
    if _typer == "field":
        _value = node.get('value', None)
        if not _value:
            raise AttributeError
        if _value == "integer":
            print(f"AVG({node['name']})")

In [63]:
for node in example_graph.successors("price_table"):
    visitor_change(example_graph.nodes[node])

AVG(sanic)
